In [29]:
import torch 
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt 
from PIL import Image
%matplotlib inline


In [30]:
from models import *
from data import *

##### Initialize Generator

In [31]:
gen = Generator(3,3,4,64)

In [32]:
gen.load_state_dict(torch.load('./netG4.pth', map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

###### Define Image Transforms

In [33]:
transforms_ = [ transforms.Resize(int(128), Image.BICUBIC), 
        transforms.RandomCrop(128), 
        #transforms.RandomVerticalFlip(p=0),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
      ]

transform = transforms.Compose(transforms_)

In [34]:

def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().detach().numpy() + 1.0)
    return image.astype(np.uint8)



##### Image must be .JPG or .jpeg

In [35]:
path = 'Screen Shot 2019-05-09 at 11.31.36 AM.jpg' #set image path

In [36]:
@interact(
          vango=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,ukyo=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,monet=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,cezan=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          ,ident=widgets.FloatSlider(min=.01,max=1,step=.01,value=1,continuous_update=False))
def generate_image(vango,ukyo,monet,cezan,ident, path= path, flip90=False):
    generator = gen
    content = transform(Image.open(path))
    Tensor = torch.Tensor
    input_A = Tensor(1, 3, 128, 128)
    real_A = input_A.copy_(content)
    
    generated = gen({
        'content': real_A,
        'style_label': [[cezan,monet,ukyo,vango,ident]]   
    })
    im=tensor2image(generated.data)
    if flip90:
        im=im.transpose(2,1,0)
    else:
        im=im.transpose(1,2,0)
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)  
    ax.imshow(im)



interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='vango', max=1.0, step=0.01)…